## EMT scoring and sample grouping (E / Hybrid / M)
- Using 76 GS & KS method

**Input from notebook 3:**
-   data/processed/tcga_brca_expression_common.tsv
-   data/processed/geo_brca_expression_common.tsv

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT = Path.home() / "emt_network"
RESULTS = PROJECT / "results"
RESULTS.mkdir(exist_ok=True)

tcga = pd.read_csv(PROJECT/"data/processed/tcga_brca_expression_common.tsv", sep="\t", index_col=0)
geo  = pd.read_csv(PROJECT/"data/processed/geo_brca_expression_common.tsv",  sep="\t", index_col=0)

tcga.shape, geo.shape


((341, 1218), (346, 3273))

In [2]:
#KS method:
from scipy.stats import ks_2samp
import pandas as pd

def ks_score(expr_genes_x_samples: pd.DataFrame, ks_excel_path) -> pd.Series:
    sig = pd.read_excel(ks_excel_path, header=None)
    sig.columns = ["gene", "class"]
    sig["gene"] = sig["gene"].astype(str).str.strip()
    sig["class"] = sig["class"].astype(str).str.strip()

    # keep only genes present in expression
    sig = sig[sig["gene"].isin(expr_genes_x_samples.index)]

    epi_genes = sig.loc[sig["class"].str.lower() == "epi", "gene"].tolist()
    mes_genes = sig.loc[sig["class"].str.lower() == "mes", "gene"].tolist()

    scores = []
    for sample in expr_genes_x_samples.columns:
        mes_vals = expr_genes_x_samples.loc[mes_genes, sample].astype(float).values
        epi_vals = expr_genes_x_samples.loc[epi_genes, sample].astype(float).values

        
        grt = ks_2samp(mes_vals, epi_vals, alternative="greater")  # Mes > Epi
        les = ks_2samp(epi_vals, mes_vals, alternative="greater")  # Epi > Mes

        if grt.pvalue < 0.05:
            score = -grt.statistic
        elif les.pvalue < 0.05:
            score = les.statistic
        else:
            # pick whichever statistic is larger; sign follows the direction
            score = les.statistic if les.statistic >= grt.statistic else -grt.statistic

        scores.append(score)

    return pd.Series(scores, index=expr_genes_x_samples.columns, name="KS_score")


In [3]:
#76 GS method:
import pandas as pd

def gs76_score(expr_genes_x_samples: pd.DataFrame, gs76_excel_path) -> pd.Series:
    sig = pd.read_excel(gs76_excel_path)
    genes = sig.iloc[:, 1].astype(str).str.strip().dropna().unique().tolist()  
    genes = [g for g in genes if g in expr_genes_x_samples.index]

    if "CDH1" not in expr_genes_x_samples.index:
        return pd.Series(0.0, index=expr_genes_x_samples.columns, name="GS76_score")

    sub = expr_genes_x_samples.loc[genes].astype(float)
    if "CDH1" not in sub.index:
        return pd.Series(0.0, index=expr_genes_x_samples.columns, name="GS76_score")

    cdh1 = sub.loc["CDH1"]

    # weight for each gene = correlation with CDH1 across samples
    weights = sub.apply(lambda row: row.corr(cdh1), axis=1).fillna(0.0)

    score = (sub.mul(weights, axis=0)).sum(axis=0)
    score = score - score.mean()
    score.name = "GS76_score"
    return score


In [4]:

ks_path = PROJECT / "resources/signatures/EM_gene_signature_tumor_KS.xlsx"
gs76_path = PROJECT / "resources/signatures/EMT_signature_76GS.xlsx"

tcga_scores = pd.concat([ks_score(tcga, ks_path), gs76_score(tcga, gs76_path)], axis=1)
geo_scores  = pd.concat([ks_score(geo, ks_path),  gs76_score(geo, gs76_path)], axis=1)

print("TCGA scores shape:", tcga_scores.shape)
print("GEO scores shape:", geo_scores.shape)

tcga_scores.head()


TCGA scores shape: (1218, 2)
GEO scores shape: (3273, 2)


,KS_score,GS76_score
TCGA-AR-A5QQ-01,0.181655,-9.901768
TCGA-D8-A1JA-01,-0.275651,7.569697
TCGA-BH-A0BQ-01,-0.139046,4.728301
TCGA-BH-A0BT-01,-0.255867,7.325293
TCGA-A8-A06X-01,-0.265759,7.650563


In [5]:
geo_scores.head()

,KS_score,GS76_score
F1,-0.097111,1.379694
F2,-0.119714,3.154668
F3,-0.167791,2.119312
F4,-0.131785,6.460251
F5,0.149143,-3.775056


In [6]:
from scipy.stats import spearmanr, pearsonr

def corr_report(df, name):
    x = df["KS_score"].astype(float).values
    y = df["GS76_score"].astype(float).values
    pear = pearsonr(x, y)
    spear = spearmanr(x, y)
    print(f"{name} Pearson r = {pear.statistic:.3f}, p = {pear.pvalue:.2e}")
    print(f"{name} Spearman rho = {spear.statistic:.3f}, p = {spear.pvalue:.2e}")

corr_report(tcga_scores, "TCGA")
corr_report(geo_scores, "GEO")


TCGA Pearson r = -0.741, p = 2.20e-212
TCGA Spearman rho = -0.811, p = 2.06e-285
GEO Pearson r = -0.825, p = 0.00e+00
GEO Spearman rho = -0.888, p = 0.00e+00


In [7]:
from scipy.stats import spearmanr, pearsonr

def corr_report(df, name):
    # use raw scores first
    x_raw = df["KS_score"].astype(float).values
    y = df["GS76_score"].astype(float).values

    pear_raw = pearsonr(x_raw, y)
    spear_raw = spearmanr(x_raw, y)

    # if KS is opposite direction to 76GS, flip it
    flipped = False
    if pear_raw.statistic < 0:
        flipped = True
        x = (-df["KS_score"]).astype(float).values
    else:
        x = x_raw

    pear = pearsonr(x, y)
    spear = spearmanr(x, y)

    print(f"{name} Pearson r (raw) = {pear_raw.statistic:.3f}, p = {pear_raw.pvalue:.2e}")
    print(f"{name} Spearman rho (raw) = {spear_raw.statistic:.3f}, p = {spear_raw.pvalue:.2e}")
    print(f"{name} KS flipped? {flipped}")
    print(f"{name} Pearson r (after flip if needed) = {pear.statistic:.3f}, p = {pear.pvalue:.2e}")
    print(f"{name} Spearman rho (after flip if needed) = {spear.statistic:.3f}, p = {spear.pvalue:.2e}")

corr_report(tcga_scores, "TCGA")
corr_report(geo_scores, "GEO")


TCGA Pearson r (raw) = -0.741, p = 2.20e-212
TCGA Spearman rho (raw) = -0.811, p = 2.06e-285
TCGA KS flipped? True
TCGA Pearson r (after flip if needed) = 0.741, p = 2.20e-212
TCGA Spearman rho (after flip if needed) = 0.811, p = 2.06e-285
GEO Pearson r (raw) = -0.825, p = 0.00e+00
GEO Spearman rho (raw) = -0.888, p = 0.00e+00
GEO KS flipped? True
GEO Pearson r (after flip if needed) = 0.825, p = 0.00e+00
GEO Spearman rho (after flip if needed) = 0.888, p = 0.00e+00


In [8]:
def add_consensus(scores: pd.DataFrame) -> pd.DataFrame:
    out = scores.copy()

    # ranks within cohort (low->high)
    out["rank_KS"] = out["KS_score"].rank(ascending=True, method="average")
    out["rank_GS76"] = out["GS76_score"].rank(ascending=True, method="average")

    out["consensus_rank"] = out[["rank_KS", "rank_GS76"]].mean(axis=1)

    q1 = out["consensus_rank"].quantile(1/3)
    q2 = out["consensus_rank"].quantile(2/3)

    out["emt_group"] = pd.cut(
        out["consensus_rank"],
        bins=[-float("inf"), q1, q2, float("inf")],
        labels=["E", "Hybrid", "M"],
        include_lowest=True
    )
    return out

tcga_final = add_consensus(tcga_scores)
geo_final  = add_consensus(geo_scores)

tcga_final["emt_group"].value_counts(), geo_final["emt_group"].value_counts()


(emt_group
 Hybrid    407
 E         406
 M         405
 Name: count, dtype: int64,
 emt_group
 E         1091
 Hybrid    1091
 M         1091
 Name: count, dtype: int64)

In [9]:
tcga_out = RESULTS / "emt_scores_tcga.csv"
geo_out  = RESULTS / "emt_scores_geo.csv"

tcga_final.to_csv(tcga_out)
geo_final.to_csv(geo_out)

tcga_out, geo_out


(PosixPath('/home/sameeksha/emt_network/results/emt_scores_tcga.csv'),
 PosixPath('/home/sameeksha/emt_network/results/emt_scores_geo.csv'))

In [10]:


reports = PROJECT / "reports"
reports.mkdir(exist_ok=True)

summary = f"""#  EMT scoring summary

## Cohorts
- TCGA BRCA: {tcga_scores.shape[0]} samples, {tcga.shape[0]} genes
- GEO GSE96058: {geo_scores.shape[0]} samples, {geo.shape[0]} genes

## Correlation (KS vs 76GS)
- TCGA Pearson r = -0.741; Spearman rho = -0.811
- GEO  Pearson r = -0.831; Spearman rho = -0.897

## EMT group counts (consensus rank)
TCGA:
{tcga_final["emt_group"].value_counts().to_string()}

GEO:
{geo_final["emt_group"].value_counts().to_string()}

## 
- results/emt_scores_tcga.csv
- results/emt_scores_geo.csv
"""

out = reports / "summary.md"
out.write_text(summary)
out


PosixPath('/home/sameeksha/emt_network/reports/summary.md')